> # 7 합성곱 신경망(CNN)

- 이미지 인식과 음성 인식 등 다양하 곳에 사용

## 7.1 전체 구조

- CNN도 기존 신경망 처럼 계층을 조합하여 만들 수 있음
- 합성곱 계층, 풀링 계층
- 기존의 완전연결 모형을 Affine 계층이라고 하자.

#### 기존 완전연결 계층(Affine 계층)

- (Affine - ReLU) $\rightarrow$ (Affine - ReLU) $\rightarrow$ (Affine - ReLU) ...
- 출력에서는 $\rightarrow$ (Affine - Softmax)

#### CNN으로 이뤄진 네트워크

- (Conv - ReLU - Pooling) $\rightarrow$ (Conv - ReLU - Pooling) $\rightarrow$ (Conv - ReLU - Pooling) $\rightarrow$ ...
- 출력에서는 마찬가지로 $\rightarrow$ (Affine - Softmax)
- 출력층에 가까워질 때 (Affine - ReLU) 조합 사용할 수 있음

## 7.2 합성곱 계층

- 패딩, 스트라이드
- 각 계층 사이에 3차원 데이터같이 입체적인 데이터가 흐름

### 7.2.1 완전연결 계층의 문제점

- 데이터의 형상이 무시됨
- 고차원 데이터를 1차원 데이터로 평탄화 하여 입력

#### CNN에서의 입출력 데이터

- 특징 맵(feature map)
- 합성곱 계층의 입력 데이터를 입력 특징 맵
- 합성곱 계층의 출력 데이터를 출력 특징 맵

### 7.2.2 합성곱 연산

- 이미지 처리에서의 필터 연산
- 문헌에 따라 필터를 커널이라고 칭하기도 함
- 윈도우를 일정 간격으로 이동해가며 입력 데이터를 적용
- 입력 데이터 - 필터 간 단일 곱셈 누산
- 마지막에 원소 각각 편향을 더해줌

### 7.2.3 패딩

- 합성곱 연산을 하기 전 데이터 주변을 특정 값으로 채우는 것
- 1로 지정하면 입력 데이터 사방을 폭이 1인 패딩 적용
- 주로 출력 크기를 조정할 목적으로 사용

### 7.2.4 스트라이드

- 필터를 적용하는 위치의 간격
- 스트라이드를 2로 하면 윈도우가 두 칸씩 이동
- 패딩, 스트라이드의 출력 크기에 대한 관계식
- 입력 크기(H, W), 필터 크기(FH, FW), 출력 크기(OH, OW), 패딩(P), 스트라이드(S)

## $OH = \frac{H + 2P - FH}{S} + 1$

## $OW = \frac{W + 2P - FW}{S} + 1$

- 각각이 정수로 나눠떨어지는 값이어야 함

### 7.2.5 3차원 데이터의 합성곱 연산

- 특징 맵이 여러 개 있다면 입력 데이터와 필터의 합성곱 연산을 채널마다 수행
- 그 결과를 더해서 하나의 출력을 얻음
- 입력 데이터의 채널 수와 필터의 채널 수가 같아야 함

### 7.2.6 블록으로 생각하기

- 3차원의 합성곱 연산은 데이터와 필터를 직육면체 블록이라고 생각하면 쉬움
- 출력으로 다수의 채널을 내보내려면? $\rightarrow$ 필터를 다수 사용
- 필터의 데이터는 4차원 (출력 채널 수, 입력 채널 수, 높이, 너비)

### 7.2.7 배치 처리

- 각 계층을 흐르는 데이터의 차원을 하나 늘려 4차원 데이터로 저장
- 데이터 수, 채널 수, 높이, 너비
- 데이터 수인 N회 분의 처리를 한 번에 수행

## 7.3 풀링 계층

- 가로 세로 방향의 공간을 줄이는 연산
- 최대 풀링은 대상 영역에서 최댓값을 구하는 방법
- 최대 풀링 외에도 평균 풀링 등이 있음

### 7.3.1 풀링 계층의 특징

- 학습해야 할 매개변수가 없다.
- 채널 수가 변하지 않는다.
- 입력의 변화에 영향을 적게 받는다.

## 7.4 합성곱/풀링 계층 구현하기

- 합성곱 계층과 풀링 계층은 복잡해 보이지만 쉽게 구현 가능

### 7.4.1 4차원 배열

#### 4차원 배열의 구현

In [2]:
import numpy as np

x = np.random.rand(10, 1, 28, 28)
x.shape

(10, 1, 28, 28)

In [5]:
print(x[0].shape, x[1].shape)

(1, 28, 28) (1, 28, 28)


In [8]:
print(x[0, 0], x[0][0])

[[0.75654238 0.80083207 0.44634959 0.25176689 0.25271202 0.38541794
  0.20708131 0.44181705 0.10785185 0.37955511 0.07818982 0.86732185
  0.81600996 0.79727244 0.89277473 0.82678589 0.81131027 0.45077007
  0.26365372 0.12283684 0.58534913 0.47508559 0.23814508 0.00704185
  0.00749462 0.78826081 0.95682265 0.41223177]
 [0.79057995 0.59380762 0.59355377 0.97503859 0.23912571 0.99883103
  0.3576766  0.34202655 0.79629833 0.21701427 0.4210945  0.12295896
  0.56906124 0.53059562 0.84975365 0.4531191  0.30200252 0.52319626
  0.05375527 0.87474714 0.07857546 0.95464908 0.34321661 0.88168165
  0.32418409 0.71473896 0.87775733 0.74704475]
 [0.23855508 0.3619455  0.3174614  0.08572451 0.00820965 0.6400362
  0.70629334 0.7194377  0.0905474  0.74574328 0.25569046 0.67060467
  0.01241988 0.96148516 0.30804767 0.55097409 0.83221504 0.08698364
  0.25101898 0.03625756 0.25039711 0.08646973 0.62235351 0.78403042
  0.21153375 0.66232854 0.58050296 0.53010778]
 [0.9225963  0.56886897 0.57673588 0.3810775

### 7.4.2 im2col로 데이터 전개하기

- 'image to column'
- im2col은 데이터를 필터링(가중치 계산)하기 좋게 전개하는 함수
- 3차원 데이터에 해당 함수를 적용하면 2차원 행렬로 바뀜
- 필터의 적용 영역이 겹치면 전개한 후의 원소 수가 원래 원소 수보다 많아짐
- 메모리를 더 많이 소비하나 행렬 곱셈을 빠르게 진행할 수 있음
- 입력 $\rightarrow$ im2col로 전개한 행렬의 내적 $\rightarrow$ 출력데이터(2차원) $\rightarrow$ 출력 데이터 reshape

### 7.4.3 합성곱 계층 구현하기

- im2col(input_data, filter_h, filter_w, stride=1, pad=0)
- input_data: (데이터 수, 채널 수, 높이, 너비)의 4차원 배열의 입력 데이터
- filter_h: 필터의 높이
- filter_w: 필터의 너비
- stride: 스트라이드
- pad: 패딩

#### im2col의 실제 사용

In [9]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

In [10]:
x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(9, 75)
(90, 75)


- 첫 번째 차원의 원소는 배치의 크기에 비례
- 모두 두 번째 차원의 원소는 75개(필터의 원소 수와 같다.)

#### 합성곱 계층 구현

In [11]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

- 합성곱 계층은 필터(가중치), 편향, 스트라이드, 패딩을 인수로 받아 초기화
- 필터는 (FN, C, FH, FW)의 4차원 형상
- transpose는 지정한 다차원 배열의 인덱스로 축 순서를 바꿔줌 .T와 같은 역할

#### 합성곱 계층의 역전파 주의사항

- im2col을 역으로 처리해야 함
- col2im 함수 사용 (common/util.py에 있음)

### 7.4.4 풀링 계층 구현하기

- 풀링 계층의 구현도 마찬가지로 im2col을 사용해 입력 데이터 전개
- 합성곱 계층과 다른 점은 채널 쪽이 독립적이라는 점

In [15]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        # 전개 (1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        # 최댓값 (2)
        out = np.max(col, axis=1)
        
        # 성형 (3)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

- 입력 데이터 전개
- 행별 최댓값 구함
- 적절한 모양 성형
- 풀링 계층의 전체 구현은 common/layers.py

## 7.5 CNN 구현하기

- 단순한 CNN 네트워크 구성
- (Conv - ReLU - Pooling) $\rightarrow$ (Affine - ReLU) $\rightarrow$ (Affine - Softmax)